In [1]:
# header files needed
import numpy as np
import torch
import torch.nn as nn
import torchvision
import random
from PIL import Image
import glob

In [2]:
# ensure the experiment produces same result on each run
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# transforms
input_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((512, 512)),    
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

target_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((512, 512)),
    torchvision.transforms.ToTensor()
])

In [ ]:
# dataset
train_dataset = torchvision.datasets.VOCSegmentation("/content/drive/My Drive/", year='2012', image_set='train', download=False, transform=input_transform, target_transform=target_transform)
val_dataset = torchvision.datasets.VOCSegmentation("/content/drive/My Drive/", year='2012', image_set='val', download=False, transform=input_transform, target_transform=target_transform)

In [ ]:
# data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=True, num_workers=8)

In [ ]:
# loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.CrossEntropyLoss(ignore_index=255)

In [ ]:
# hyper-parameters and hard-coded values
num_classes = 21
num_epochs = 100
lr = 1e-5
wd = 1e-5

In [ ]:
class SegNet(torch.nn.Module):

  def __init__(self, pretrained_net, num_classes=21):
    super(SegNet, self).__init__()

    # encoder 1, encoder 2, encoder 3, encoder 4, encoder 5
    self.encoder_block_1 = torch.nn.Sequential(*list(model.features.children())[:-38])
    self.encoder_block_2 = torch.nn.Sequential(*list(model.features.children())[-37:-31])
    self.encoder_block_3 = torch.nn.Sequential(*list(model.features.children())[-30:-21])
    self.encoder_block_4 = torch.nn.Sequential(*list(model.features.children())[-20:-11])
    self.encoder_block_5 = torch.nn.Sequential(*list(model.features.children())[-10:-1])

    # max-pool layer with return_indices as true
    self.max_pool_layer = torch.nn.Sequential(
        torch.nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
    )

    # decoder block 1
    self.decoder_block_1 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(512, 512, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(512),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(512, 512, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(512),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(512, 512, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(512),
        torch.nn.ReLU(inplace=True)
    )

    # decoder block 2
    self.decoder_block_2 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(512, 512, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(512),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(512, 512, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(512),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(512, 256, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(256),
        torch.nn.ReLU(inplace=True)
    )

    # decoder block 3
    self.decoder_block_3 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(256, 256, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(256),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(256, 256, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(256),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(256, 128, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(128),
        torch.nn.ReLU(inplace=True)
    )

    # decoder block 4
    self.decoder_block_4 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(128, 128, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(128),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(64),
        torch.nn.ReLU(inplace=True)
    )

    # decoder block 5
    self.decoder_block_5 = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1),
        torch.nn.BatchNorm2d(64),
        torch.nn.ReLU(inplace=True),
        torch.nn.ConvTranspose2d(64, num_classes, kernel_size=3, padding=1)
    )

  def forward(self, x):
    
    # apply encoder block 1
    enc_1 = self.encoder_block_1(x)
    enc_1, m_1 = self.max_pool_layer(enc_1)

    # apply encoder block 2
    enc_2 = self.encoder_block_2(enc_1)
    enc_2, m_2 = self.max_pool_layer(enc_2)

    # apply encoder block 3
    enc_3 = self.encoder_block_3(enc_2)
    enc_3, m_3 = self.max_pool_layer(enc_3)

    # apply encoder block 4
    enc_4 = self.encoder_block_4(enc_3)
    enc_4, m_4 = self.max_pool_layer(enc_4)

    # apply encoder block 5
    enc_5 = self.encoder_block_5(enc_4)
    enc_5, m_5 = self.max_pool_layer(enc_5)

    # apply decoder block 1
    dec_1 = self.decoder_block_1(torch.nn.functional.max_unpool2d(enc_5, m_5, kernel_size=2, stride=2, output_size=enc_4.size()))

    # apply decoder block 2
    dec_2 = self.decoder_block_2(torch.nn.functional.max_unpool2d(enc_4, m_4, kernel_size=2, stride=2, output_size=enc_3.size()))

    # apply decoder block 3
    dec_3 = self.decoder_block_3(torch.nn.functional.max_unpool2d(enc_3, m_3, kernel_size=2, stride=2, output_size=enc_2.size()))

    # apply decoder block 4
    dec_4 = self.decoder_block_4(torch.nn.functional.max_unpool2d(enc_2, m_2, kernel_size=2, stride=2, output_size=enc_1.size()))

    output = self.decoder_block_5(torch.nn.functional.max_unpool2d(dec_4, m_1, kernel_size=2, stride=2, output_size=x.size()))
    return output

In [ ]:
model = torchvision.models.vgg16_bn(pretrained=True)
model = SegNet(model, num_classes)
model.to(device)

In [ ]:
# create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [ ]:
train_loss_list = []
train_accuracy_list = []
train_iou_list = []
val_loss_list = []
val_accuracy_list = []
val_iou_list = []

# training and val loop
for epoch in range(0, num_epochs):

  # train
  model.train()
  train_loss = 0.0
  train_accuracy = 0.0
  train_iou = 0.0
  for step, (images, labels) in enumerate(train_loader):
    
    # if cuda
    images = images.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.reshape(labels.shape[0], labels.shape[2], labels.shape[3])
    labels = labels.to(device)
    
    # get loss
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

    # convert outputs and labels to rank-1 tensor
    outputs = outputs.argmax(1).flatten().cpu()
    labels = labels.flatten().cpu()
    intersection = outputs * (outputs == labels).long()

    # update training_accuracy
    total = ((labels == labels) * (labels > 0)).sum()
    correct = ((outputs == labels) * (labels > 0)).sum()
    train_accuracy += np.round(float(float(correct) / float(total)), 6)

    # update training_iou
    area_intersection = torch.histc(intersection.float(), bins=num_classes-1, max=num_classes-1, min=1)
    area_labels = torch.histc(labels.float(), bins=num_classes-1, max=num_classes-1, min=1)
    area_outputs = torch.histc(outputs.float(), bins=num_classes-1, max=num_classes-1, min=1)
    area_union = area_outputs + area_labels - area_intersection
    train_iou += np.round(float(float(torch.sum(area_intersection)) / float(torch.sum(area_union))), 6)

  # update training_loss, training_accuracy and training_iou 
  train_loss = train_loss / float(len(train_loader))
  train_accuracy = train_accuracy / float(len(train_loader))
  train_iou = train_iou / float(len(train_loader))
  train_loss_list.append(train_loss)
  train_accuracy_list.append(train_accuracy)
  train_iou_list.append(train_iou)

  
  # evaluation code
  model.eval()
  val_loss = 0.0
  val_accuracy = 0.0
  val_iou = 0.0
  for step, (images, labels) in enumerate(val_loader):
    with torch.no_grad():

      # if cuda
      images = images.to(device)
      labels = labels.type(torch.LongTensor)
      labels = labels.reshape(labels.shape[0], labels.shape[2], labels.shape[3])
      labels = labels.to(device)

      # get loss
      outputs = model(images)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      # convert outputs and labels to rank-1 tensor
      outputs = outputs.argmax(1).flatten().cpu()
      labels = labels.flatten().cpu()
      intersection = outputs * (outputs == labels).long()

      # update val_accuracy
      total = ((labels == labels) * (labels > 0)).sum()
      correct = ((outputs == labels) * (labels > 0)).sum()
      val_accuracy += np.round(float(float(correct) / float(total)), 4)

      # update val_iou
      area_intersection = torch.histc(intersection.float(), bins=num_classes-1, max=num_classes-1, min=1)
      area_labels = torch.histc(labels.float(), bins=num_classes-1, max=num_classes-1, min=1)
      area_outputs = torch.histc(outputs.float(), bins=num_classes-1, max=num_classes-1, min=1)
      area_union = area_outputs + area_labels - area_intersection
      val_iou += np.round(float(float(torch.sum(area_intersection)) / float(torch.sum(area_union))), 6)

  # update val_loss, val_accuracy and val_iou 
  val_loss = val_loss / float(len(val_loader))
  val_accuracy = val_accuracy / float(len(val_loader))
  val_iou = val_iou / float(len(val_loader))
  val_loss_list.append(val_loss)
  val_accuracy_list.append(val_accuracy)
  val_iou_list.append(val_iou)


  print()
  print("Epoch: " + str(epoch))
  print("Training Loss: " + str(train_loss) + "    Validation Loss: " + str(val_loss))
  print("Training Accuracy: " + str(train_accuracy) + "    Validation Accuracy: " + str(val_accuracy))
  print("Training mIoU: " + str(train_iou) + "    Validation mIoU: " + str(val_iou))
  print()

In [ ]:
import matplotlib.pyplot as plt
e = []
for index in range(0, num_epochs):
  e.append(index)

In [ ]:
plt.plot(e, train_loss_list)
plt.show()

In [ ]:
plt.plot(e, val_loss_list)
plt.show()